This is my first try to train a CNN network. Planed to use an Alex net, but found that it takes too long to train it on CPU. So I gived up and change to a LeNet-like CNN to get result faster. I started working on this on Sunday morning(2.26), took a day to complete

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from keras.preprocessing import image
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Convolution2D, MaxPooling2D, Flatten, Input

# Reading data
Read data from data_folder, and transform it into form demanded by keras

In [ ]:
data_root = "../input/"
img_folder = data_root + 'images/'
train_data = pd.read_csv(data_root + 'train.csv')
train_ID = train_data['id']
train_Y = train_data['species']
test_data = pd.read_csv(data_root + 'test.csv')
test_ID = test_data['id']

le = LabelEncoder()
train_y = le.fit_transform(train_Y)

# Preprocess img
Downsize the img and change it into grayscale binary format to save memory.
Centrelized the image data on the input matrices. This part I took [kaggle notebook][1] as reference.


  [1]: https://www.kaggle.com/abhmul/leaf-classification/keras-convnet-lb-0-0052-w-visualization

In [ ]:
def resize_img(img, max_dim=96):
    large_axis = max((0, 1), key=lambda x: img.size[x])
    scalar = max_dim / float(img.size[large_axis])
    resized = img.resize(
        (int(img.size[0] * scalar), int(img.size[1] * scalar)))
    return resized


def load_image_data(id_list, max_dim=96, center=True):
    X = np.empty((len(id_list), max_dim, max_dim, 1))
    for i, idnum in enumerate(id_list):
        x = image.load_img(
            (img_folder + str(idnum) + '.jpg'), grayscale=True)
        x = image.img_to_array(resize_img(x, max_dim=max_dim))
        height = x.shape[0]
        width = x.shape[1]
        if center:
            h1 = int((max_dim - height) / 2)
            h2 = h1 + height
            w1 = int((max_dim - width) / 2)
            w2 = w1 + width
        else:
            h1, w1 = 0, 0
            h2, w2 = (height, width)
        X[i, h1:h2, w1:w2, :] = x
    return np.around(X / 255.0)

# Build the Alexnet
Build the Alexnet model based on descrption in [This blog (In Chinese)][1]


  [1]: http://blog.csdn.net/sunbaigui/article/details/39938097

In [ ]:
def AlexNex(input_layer):
    conv_1 = Convolution2D(96, 11, 11, activation='relu', input_shape=(
        96, 96, 1), border_mode='same', name='conv1')(input_layer)
    max_pool_1 = MaxPooling2D((3, 3), strides=(2, 2))(conv_1)

    conv_2 = Convolution2D(256, 5, 5, border_mode='same',
                           activation='relu')(max_pool_1)
    max_pool_2 = MaxPooling2D((3, 3), strides=(2, 2))(conv_2)

    conv_3 = Convolution2D(384, 3, 3, border_mode='same',
                           activation='relu')(max_pool_2)
    conv_4 = Convolution2D(384, 3, 3, border_mode='same',
                           activation='relu')(conv_3)

    conv_5 = Convolution2D(256, 3, 3, border_mode='same',
                           activation='relu')(conv_4)
    max_pool_5 = MaxPooling2D((3, 3), strides=(2, 2))(conv_5)

    flat = Flatten()(max_pool_5)
    dense_1 = Dense(4096, init='glorot_normal', activation='relu')(flat)
    drop_1 = Dropout(0.5)(dense_1)

    dense_2 = Dense(4096, init='glorot_normal', activation='relu')(drop_1)
    drop_2 = Dropout(0.5)(dense_2)

    output_layer = Dense(99, activation='softmax')(drop_2)

    model = Model(input_layer, output_layer)
    return model

But the training process is to slow, I start this on Sunday afternoon, and it took about 500 sec to train a epoch.

    990/990 [==============================] - 548s - loss: 0.8667 - acc: 0.6970    
    Epoch 17/50
    128/990 [==>...........................] - ETA: 480s - loss: 0.7794 - acc: 0.7656
    256/990 [======>.......................] - ETA: 408s - loss: 0.7746 - acc: 0.7617
    384/990 [==========>...................] - ETA: 336s - loss: 0.7541 - acc: 0.7682
    512/990 [==============>...............] - ETA: 265s - loss: 0.7583 - acc: 0.7715
    640/990 [==================>...........] - ETA: 193s - loss: 0.7449 - acc: 0.7781[Cancelled]

And 50 epoch seems not enough to convergent, so I gived up halfway and build another LeNet-like model

In [ ]:
def NaiveCovNet(input_layer):
    x = Convolution2D(8, 5, 5, input_shape=(96, 96, 1),
                      border_mode='same')(input_layer)
    x = (Activation('relu'))(x)
    x = (MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))(x)

    # Now through the second convolutional layer
    x = (Convolution2D(32, 5, 5, border_mode='same'))(x)
    x = (Activation('relu'))(x)
    x = (MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))(x)

    # Flatten our array
    x = Flatten()(x)
    dense_1 = Dense(1024, init='glorot_normal', activation='relu')(x)
    drop_1 = Dropout(0.5)(dense_1)

    dense_2 = Dense(99, init='glorot_normal', activation='relu')(drop_1)
    drop_2 = Dropout(0.5)(dense_2)

    output_layer = Dense(99, activation='softmax')(drop_2)
    model = Model(input_layer, output_layer)
    return model

In [ ]:
input_layer = Input(shape=(96, 96, 1), name='image')
model = NaiveCovNet(input_layer)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

trian_X = load_image_data(train_ID)
train_y = to_categorical(train_y)

history = model.fit(trian_X, train_y, nb_epoch=10, batch_size=128)
#f_model = './model'
#json_string = model.to_json()
#open(os.path.join(f_model, 'model.json'), 'w').write(json_string)
#print('save weights')
#model.save_weights(os.path.join(f_model, 'model_weights.hdf5'))

This model took about 20 min to trian for 100 epoch, and get such result:

    Epoch 91/100
    990/990 [==============================] - 16s - loss: 0.0933 - acc: 0.9727
    Epoch 92/100
    990/990 [==============================] - 17s - loss: 0.0869 - acc: 0.9768
    Epoch 93/100
    990/990 [==============================] - 16s - loss: 0.0849 - acc: 0.9737
    Epoch 94/100
    990/990 [==============================] - 17s - loss: 0.0925 - acc: 0.9727
    Epoch 95/100
    990/990 [==============================] - 17s - loss: 0.0859 - acc: 0.9747
    Epoch 96/100
    990/990 [==============================] - 17s - loss: 0.0858 - acc: 0.9758
    Epoch 97/100
    990/990 [==============================] - 17s - loss: 0.0662 - acc: 0.9798
    Epoch 98/100
    990/990 [==============================] - 17s - loss: 0.0896 - acc: 0.9758
    Epoch 99/100
    990/990 [==============================] - 16s - loss: 0.0763 - acc: 0.9737
    Epoch 100/100
    990/990 [==============================] - 17s - loss: 0.0725 - acc: 0.9778
    save weights

It did prety well on training set so I save the weight for further use.

Predict and save result

In [ ]:
#from keras.models import model_from_json
#f_model = './model'
#model_filename = 'cnn_model.json'
#weights_filename = 'cnn_model_weights.hdf5'
#json_string = open(os.path.join(f_model, model_filename)).read()
#model = model_from_json(json_string)

#model.summary()

#model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.load_weights(os.path.join(f_model,weights_filename))

In [ ]:
X_test = load_image_data(test_ID)
CNN_pred = model.predict(X_test)
LABELS = sorted(pd.read_csv(os.path.join(data_root, 'train.csv')).species.unique())
save_File = pd.DataFrame(CNN_pred,index=test_ID,columns=LABELS)
save_File.to_csv("submission.csv", index=True)

This is my first time try out CNN method, the result is not ideal, and I plan to improve it with the following method:

 - Apply data augment to training set. 990 examples is far from enough,
   maybe I should try keras pre-built method ImageDataGenerator. 
 - Use a machine with GPU and use model like VGG-16 net or Resnet. 
 - I have found many pretrained model based on imagenet on Internet. Modify some top layers and fin-tuned them could have better output.
 - Use CNN to extract feathers and combined it with other manully selected feather in dataset.

This is just a learning process of buliding and training CNN network so I didn't focus on getting better socre and utlizing avalible data.